# Generate pressure coefficients

Initialize Path Manager

In [13]:
from cfdmod.use_cases.pressure.path_manager import CpPathManager, copy_input_artifacts
import pathlib

path_manager = CpPathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Cp config file

In [14]:
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig

cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml")
post_proc_cfg = CpCaseConfig.from_file(cfg_path)

post_proc_cfg

CpCaseConfig(pressure_coefficient={'default': CpConfig(number_of_chunks=10, timestep_range=(10000.0, 20000.0), reference_pressure='average', U_H=0.05, U_H_correction_factor=1, statistics=['min', 'max', 'std', 'mean', 'skewness', 'kurtosis', 'xtr_min', 'xtr_max', 'mean_eq'])}, extreme_values=ExtremeValuesParameters(CST_real=0.56, CST_sim=16.89, extreme_model='Moving average', parameters={'window_size_real': 3}, time_scale_correction_factor=0.61))

Read LNAS Mesh

In [15]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Read hist series

In [16]:
import pandas as pd

static_data_path = pathlib.Path(
    "./fixtures/tests/pressure/data/points.static_pressure.data.resampled.h5"
)
body_data_path = pathlib.Path("./fixtures/tests/pressure/data/bodies.galpao.data.resampled.h5")

Process pressure coefficient use case

In [17]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.cp_data import process_cp
from cfdmod.use_cases.pressure.path_manager import copy_input_artifacts

for cfg_lbl, cfg in post_proc_cfg.pressure_coefficient.items():
    logger.info(f"Processing pressure coefficients for config {cfg_lbl} ...")
    process_cp(
        pressure_data_path=static_data_path,
        body_data_path=body_data_path,
        cfg_label=cfg_lbl,
        cfg=cfg,
        mesh=mesh.geometry,
        path_manager=path_manager,
        extreme_params=post_proc_cfg.extreme_values,
    )

    logger.info("Copying input artifacts")
    copy_input_artifacts(
        cfg_path=cfg_path,
        mesh_path=mesh_path,
        static_data_path=static_data_path,
        body_data_path=body_data_path,
        path_manager=path_manager,
    )

    logger.info(f"Processed pressure coefficients for config {cfg_lbl}!")

[2024-04-18 11:17:59,358] [INFO] - cfdmod - Processing pressure coefficients for config default ... (891451574.py:6)
[2024-04-18 11:17:59,361] [INFO] - cfdmod - Transforming into pressure coefficient (cp_data.py:164)
[2024-04-18 11:17:59,419] [INFO] - cfdmod - Dividing into point groups (cp_data.py:181)
[2024-04-18 11:17:59,524] [INFO] - cfdmod - Calculating statistics (cp_data.py:188)


/home/ubuntu/Documentos/Repositories/cfdmod/cfdmod/use_cases/pressure/cp_data.py:158: RuntimeWarning: Path for time series already exists output/pressure/cp/default/dc51b167d253368f161800ada22e8cc253820d9ef7b3339281fe628c5177e492/time_series.h5. Deleted old file
  warnings.warn(
/home/ubuntu/Documentos/Repositories/cfdmod/cfdmod/use_cases/pressure/cp_data.py:175: RuntimeWarning: Path for grouped time series already exists output/pressure/cp/default/dc51b167d253368f161800ada22e8cc253820d9ef7b3339281fe628c5177e492/time_series.grouped.h5. Deleted old file
  warnings.warn(


[2024-04-18 11:18:00,722] [INFO] - cfdmod - Copying input artifacts (891451574.py:17)
[2024-04-18 11:18:00,728] [INFO] - cfdmod - Processed pressure coefficients for config default! (891451574.py:26)


Copy Input artifacts to output folder